`ppdire.py` subpackage: examples
================================
Here, we will illustrate how to use `ppdire.py` to perform different types of projection pursuit dimension reduction. 

To run a toy example, start by sourcing packages and data: 

In [1]:
# Load data
import pandas as ps
import numpy as np
data = ps.read_csv("../data/Returns_shares.csv")
columns = data.columns[2:8]
(n,p) = data.shape
datav = np.matrix(data.values[:,2:8].astype('float64'))
y = datav[:,0]
X = datav[:,1:5]
        
# Scale data
from direpack import VersatileScaler
centring = VersatileScaler()
Xs = centring.fit_transform(X)

1\. Comparison of PP estimates to Scikit-Learn 
======================================
Let us at first run `ppdire` to produce slow, approximate PP estimates of 
PCA and PLS. This makes it easy to verify that the algorithm is correct. 
        
1\.1\. PCA
--------------
By setting the projection index to variance, projection pursuit is a slow, approximate way to calculate PCA. Let's compare the `ppdire` results to `sklearn`'s.

- PCA ex `scikit-learn` 

In [2]:
import sklearn.decomposition as skd
skpca = skd.PCA(n_components=4)
skpca.fit(Xs)
skpca.components_.T # sklearn outputs loadings as rows ! 

array([[ 0.6160923 ,  0.09574799, -0.27553202,  0.73167254],
       [ 0.3612025 ,  0.79963754,  0.40566716, -0.25602091],
       [ 0.57369773, -0.2220554 , -0.47175995, -0.63166832],
       [ 0.40104108, -0.54963945,  0.73277473,  0.01018414]])

- PCA ex `ppdire`, using SLSQP optimization 

In [3]:
from direpack import dicomo, ppdire
pppca = ppdire(projection_index = dicomo, pi_arguments = {'mode' : 'var'}, n_components=4, optimizer='SLSQP')
pppca.fit(X)
pppca.x_loadings_

array([[-0.61607418,  0.09583099,  0.27564666, -0.73163593],
       [-0.36117418,  0.79968851, -0.40559672,  0.2560418 ],
       [-0.57366555, -0.22197832,  0.47186547,  0.63170265],
       [-0.40114045, -0.54958196, -0.73270267, -0.01015965]])

- PCA ex `ppdire`, using its native `grid` algorithm optimization \[1\].

In [4]:
pppca = ppdire(projection_index = dicomo, pi_arguments = {'mode' : 'var'}, n_components=4, optimizer='grid',optimizer_options={'ndir':1000,'maxiter':1000})
pppca.fit(X)
pppca.x_loadings_

1
2
3
1
2
3
4
1
2
3
4
1
2


array([[ 0.61598274, -0.09421448, -0.27613309,  0.73205157],
       [ 0.36137778, -0.80127483,  0.40173098, -0.25522632],
       [ 0.57377558,  0.22404487, -0.47051245, -0.63154901],
       [ 0.40094043,  0.54670882,  0.73551452,  0.01041449]])

1\.2\. PLS 
----------
Likewise, by setting the projection index to covariance, one obtains partial least squares. 

- PLS ex `scikit-learn`

In [5]:
import sklearn.cross_decomposition as skc
skpls = skc.PLSRegression(n_components=4)
skpls.fit(Xs,(y-np.mean(y))/np.std(y))
skpls.x_scores_
print(skpls.coef_) 
Xs*skpls.coef_*np.std(y) + np.mean(y)

[[ 0.35739958]
 [-0.04417834]
 [ 0.22635622]
 [ 0.04614664]]


matrix([[-0.27212755],
        [ 0.83959019],
        [-0.22662337],
        ...,
        [-0.03176729],
        [-0.10770676],
        [ 0.26843739]])

- PLS ex `ppdire`, using SLSQP optimization 

In [6]:
pppls = ppdire(projection_index = dicomo, pi_arguments = {'mode' : 'cov'}, n_components=4, square_pi=True, optimizer='SLSQP', optimizer_options={'maxiter':500})
pppls.fit(X,y)
pppls.x_scores_
print(pppls.coef_scaled_) # Column 4 should agree with skpls.coef_
pppls.fitted_  

[[ 0.27190446  0.33122054  0.35628291  0.35734814]
 [ 0.05506195 -0.04547853 -0.04113433 -0.04417198]
 [ 0.23907241  0.25452063  0.22888164  0.22632364]
 [ 0.10731098  0.0461689   0.04141709  0.04614   ]]


matrix([[-0.27208478],
        [ 0.83947294],
        [-0.22658715],
        ...,
        [-0.03175912],
        [-0.10768766],
        [ 0.26840235]])

- PLS ex `ppdire`, `grid` optimization

In [7]:
pppls = ppdire(projection_index = dicomo, pi_arguments = {'mode' : 'cov'}, n_components=4, square_pi=True, optimizer='grid',optimizer_options={'ndir':1000,'maxiter':1000})
pppls.fit(X,y)
pppls.x_scores_
print(pppls.coef_scaled_) # Column 4 should agree with skpls.coef_
pppls.fitted_  

1
2
1
2
1
2
1
2
[[ 0.27197566  0.33138071  0.35633098  0.35734814]
 [ 0.05543391 -0.0453769  -0.04118079 -0.04417198]
 [ 0.23857626  0.25436242  0.22879951  0.22632364]
 [ 0.10758362  0.04611615  0.04150847  0.04614   ]]


matrix([[-0.27208478],
        [ 0.83947294],
        [-0.22658715],
        ...,
        [-0.03175912],
        [-0.10768766],
        [ 0.26840235]])

Remark: Dimension Reduction techniques based on projection onto latent variables, 
such as PCA, PLS and ICA, are sign indeterminate with respect to the components. 
Therefore, signs of estimates by different algorithms can be opposed, yet the 
absolute values should be identical up to algorithm precision.  Here, this implies
that `sklearn` and `ppdire`'s `x_scores_` and `x_loadings` can have opposed signs,
yet the coefficients and fitted responses should be identical. 

2\. Robust projection pursuit estimators
=================================

Note that optimization through `scipy.optimize` is much more efficient than the native `grid` algorithm, yet will only provide correct results for classical projection indices. The native `grid` algorithm should be used when 
the projection index involves order statistics of any kind, such as ranks, trimming, winsorizing, or empirical quantiles.

- Robust PCA based on the Median Absolute Deviation (MAD) \[2\]. 

In [8]:
lcpca = ppdire(projection_index = dicomo, pi_arguments = {'mode' : 'var', 'center': 'median'}, n_components=4, optimizer='grid',optimizer_options={'ndir':1000,'maxiter':10})
#set a higher maxiter for convergence and precision! 
lcpca.fit(X)
lcpca.x_loadings_

1
2
3
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
1
2


array([[ 0.6324543 , -0.00651997, -0.35820225,  0.6438448 ],
       [ 0.44750274, -0.67228343,  0.4950862 , -0.21806968],
       [ 0.53378114,  0.28794634, -0.46650197, -0.72699245],
       [ 0.35432068,  0.68524337,  0.64350842,  0.09692107]])

To extend to Robust PCR, just add `y`: 

In [9]:
 lcpca.fit(X,y,regopt='robust')

1
2
3
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
1
2


ppdire(alpha=1, center='mean', center_data=True, compression=False, copy=True,
       n_components=4, optimizer='grid', optimizer_constraints={},
       optimizer_options={'maxiter': 10, 'ndir': 1000},
       pi_arguments={'center': 'median', 'mode': 'var'},
       projection_index=<class 'direpack.dicomo.dicomo.dicomo'>, regopt='OLS',
       return_scaling_object=True, scale_data=True, square_pi=False, trimming=0,
       verbose=True, whiten_data=False)

- Robust Continuum Regression \[4\] based on trimmed continuum association: 

In [10]:
rcr = ppdire(projection_index = dicomo, pi_arguments = {'mode' : 'continuum'}, n_components=4, trimming=.1, alpha=.5, optimizer='grid',optimizer_options={'ndir':250,'maxiter':1000})
rcr.fit(X,y=y,regopt='robust')
rcr.x_loadings_
rcr.x_scores_
rcr.coef_scaled_
rcr.predict(X)

1
2
1
2
3
1
2
3
4
5
6
7
8
9
1
2


array([-0.26770602,  0.8438178 , -0.22220975, ..., -0.02740443,
       -0.10328256,  0.27283931])

3\. Projection pursuit generalized betas
================================

Generalized betas are obtained as the projection pursuit weights using the 
co-moment analysis projection index (CAPI) \[2\]. 

In [11]:
from ppdire import capi 
est = ppdire(projection_index = capi, pi_arguments = {'max_degree' : 3,'projection_index': dicomo, 'scaling': False}, n_components=1, trimming=0,center_data=True,scale_data=True)
est.fit(X,y=y,ndir=200)
est.x_weights_

array([[ 0.89537008],
       [-0.149126  ],
       [ 0.03742233],
       [ 0.41793949]])

Note that these data aren't the greatest illustration. Evaluating CAPI projections, makes more sense if y is a market index, e.g. SPX. 

4\. Cross-validating through `scikit-learn` 
===========================================

The `ppdire` class is 100% compatible with `scikit-learn`, which allows, for instance, hyperparameter tuning through `GridSearchCV`. 
To try out, uncomment the line below and run. (this may take some time). 

In [12]:
# Uncomment to try out:

# from sklearn.model_selection import GridSearchCV
# rcr_cv = GridSearchCV(ppdire(projection_index=dicomo, 
#                                    pi_arguments = {'mode' : 'continuum'
#                                                    }, 
#                                    optimizer = 'grid',
#                                   optimizer_options = {'ndir':1000,'maxiter':1000}
#                                   ), 
#                             cv=10, 
#                             param_grid={"n_components": [1, 2, 3], 
#                                         "alpha": np.arange(.1,3,.3).tolist(),
#                                         "trimming": [0, .15]
#                                        }
#                            )
#       rcr_cv.fit(X[:2666],y[:2666]) 
#       rcr_cv.best_params_
#       rcr_cv.predict(X[2666:])

5\. Data compression
=================
While `ppdire` is very flexible and can project according to a very wide variety 
of projection indices, it can be computationally demanding. For flat data tables,
a workaround has been built in. However, not that running the code in the next field can take quite some time nonetheless.   

In [16]:
# Load flat data 
datan = ps.read_csv("../data/Glass_df.csv")
X = datan.values[:,100:300]
y = datan.values[:,2]
        
# Now compare
rcr = ppdire(projection_index = dicomo, 
                    pi_arguments = {'mode' : 'continuum'}, 
                    n_components=4, 
                    trimming=.1, 
                    alpha=.5, 
                    compression = False, 
                    optimizer='grid',
                    optimizer_options={'ndir':1000,'maxiter':1000})
rcr.fit(X,y)
print(rcr.coef_)
        
rcr = ppdire(projection_index = dicomo, 
                    pi_arguments = {'mode' : 'continuum'}, 
                    n_components=4, 
                    trimming=.1, 
                    alpha=.5, 
                    compression = True, 
                    optimizer='grid',
                    optimizer_options={'ndir':1000,'maxiter':1000})
rcr.fit(X,y)
rcr.coef_

1
2
3
4
5
6
7
1
2
3
4
5
6
7
8
1
2
3
1
2
[[-4.71006531e-05 -1.73132209e-05 -2.32341283e-05  1.92662729e-05]
 [-4.91739225e-05 -2.17947308e-05 -2.87668265e-05  1.20306899e-05]
 [-5.12759847e-05 -1.42524723e-05 -1.93896027e-05  3.01898039e-05]
 [-5.52807363e-05 -1.16097279e-05 -1.61289191e-05  3.89100248e-05]
 [-6.40861390e-05 -2.28662497e-05 -3.16943089e-05  2.28228633e-05]
 [-7.48150962e-05 -1.73256629e-05 -2.46603844e-05  4.64485391e-05]
 [-9.23330952e-05 -1.00954653e-05 -1.93254855e-05  7.29662769e-05]
 [-1.15254313e-04 -7.55476110e-06 -1.78942404e-05  9.62501844e-05]
 [-1.47591582e-04  1.82604412e-05  9.50671369e-06  1.71444482e-04]
 [-1.61291766e-04  1.09591453e-04  1.11139450e-04  3.23510785e-04]
 [-1.70482657e-04  3.03507364e-04  3.33934289e-04  6.62455264e-04]
 [-1.45179276e-04  5.85905143e-04  6.65129657e-04  1.10966356e-03]
 [-7.77104871e-05  8.64100138e-04  9.93342063e-04  1.53689877e-03]
 [ 2.11272402e-05  1.29104873e-03  1.50740200e-03  2.22823988e-03]
 [ 7.15290516e-05  1.5

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


array([[-1.71734083e-03,  9.89246839e-03,  1.03132041e-02,
         1.23950958e-02],
       [-1.18750786e-03,  7.82790568e-04,  1.56385213e-04,
         4.86985132e-03],
       [ 3.42615075e-03, -1.02674075e-03, -1.02432021e-03,
        -2.68402298e-03],
       [ 7.20459942e-03,  1.16306226e-02,  1.14398455e-02,
         1.27104444e-02],
       [-7.45358004e-03, -9.26351554e-03, -1.07302614e-02,
        -1.32082506e-02],
       [-5.35992939e-05,  4.69698462e-03,  5.35924092e-03,
         4.86090170e-03],
       [-3.99197872e-03, -2.46572486e-02, -2.43193644e-02,
        -2.73214854e-02],
       [-7.82281660e-04, -1.22765531e-02, -1.35454735e-02,
        -1.57319221e-02],
       [-4.14575962e-03, -1.20971538e-03, -1.06979268e-03,
         3.54833732e-03],
       [-4.11097072e-03,  2.33337829e-03,  4.83267762e-03,
         5.30653152e-03],
       [-1.76870631e-03, -1.16439795e-02, -1.02689563e-02,
        -1.66622433e-02],
       [ 1.59799088e-03, -2.44734705e-04, -2.91311070e-03,
      

However, compression will not work properly if the data contain several low scale 
varables. In this example, it will not work for `X = datan.values[:,8:751]`. This 
will throw a warning, and `ppdire` will continue without compression. 

References
----------------
1. [Robust Multivariate Methods: The Projection Pursuit Approach](https://link.springer.com/chapter/10.1007/3-540-31314-1_32), Peter Filzmoser, Sven Serneels, Christophe Croux and Pierre J. Van Espen, in: From Data and Information Analysis to Knowledge Engineering, Spiliopoulou, M., Kruse, R., Borgelt, C., Nuernberger, A. and Gaul, W., eds., Springer Verlag, Berlin, Germany, 2006, pages 270--277.
2. [Projection pursuit based generalized betas accounting for higher order co-moment effects in financial market analysis](https://arxiv.org/pdf/1908.00141.pdf), Sven Serneels, in: JSM Proceedings, Business and Economic Statistics Section. Alexandria, VA: American Statistical Association, 2019, 3009-3035.
3. Robust principal components and dispersion matrices via projection pursuit, Chen, Z. and Li, G., Research Report, Department of Statistics, Harvard University, 1981.
4. [Robust Continuum Regression](https://www.sciencedirect.com/science/article/abs/pii/S0169743904002667), Sven Serneels, Peter Filzmoser, Christophe Croux, Pierre J. Van Espen, Chemometrics and Intelligent Laboratory Systems, 76 (2005), 197-204.